In [6]:
import os
import json

from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy
from fabric_cf.orchestrator.orchestrator_proxy import OrchestratorProxy

#Create a FABRIC Orchestrator Proxy.
orchestrator_proxy = OrchestratorProxy(orchestrator_host=os.environ['FABRIC_ORCHESTRATOR_HOST'])

#Create a Credential Manager Proxy 
credmgr_proxy = CredmgrProxy(credmgr_host=os.environ['FABRIC_CREDMGR_HOST'])

#Cofigure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

In [7]:
#Retrieve or set the refresh token (exprires 24 hours after login)
fabric_refresh_token=None
%store -r fabric_refresh_token
if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token

print("Refresh Token: {}".format(fabric_refresh_token))

Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCYTFMI3DEOJQGJSTIZRRGA4TMMBUMYZGMZTCGYZGENDFHAZTSZR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMJZGA3TIOBRG42DCOBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


In [8]:
try:
    refresh_res = credmgr_proxy.refresh(project_name='all', scope='all', refresh_token=fabric_refresh_token)

    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print("New Refreash Token: {}\n".format(fabric_refresh_token))
    print("New ID Token: {}".format(fabric_id_token))
    %store fabric_refresh_token

except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

New Refreash Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXWGNZSGMYDCNTBMU4GCYJTMEYWENJRGQ2TAN3GMQZGGZJQGBQTMP3UPFYGKPLSMVTHEZLTNBKG623FNYTHI4Z5GE3DCOJQG42DQMZRGUZTOJTWMVZHG2LPNY6XMMROGATGY2LGMV2GS3LFHU4DMNBQGAYDAMA

New ID Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImI0MTUxNjcyMTExOTFlMmUwNWIyMmI1NGIxZDNiNzY2N2U3NjRhNzQ3NzIyMTg1ZTcyMmU1MmUxNDZmZTQzYWEifQ.eyJlbWFpbCI6Imp1bEBjbGVtc29uLmVkdSIsImNlcnRfc3ViamVjdF9kbiI6Ii9EQz1vcmcvREM9Y2lsb2dvbi9DPVVTL089Q2xlbXNvbiBVbml2ZXJzaXR5L0NOPUp1IExpbiBBNDE4Njc4OTYiLCJpZHAiOiJ1cm46bWFjZTppbmNvbW1vbjpjbGVtc29uLmVkdSIsImlkcF9uYW1lIjoiQ2xlbXNvbiBVbml2ZXJzaXR5IiwiZXBwbiI6Imp1bEBjbGVtc29uLmVkdSIsImVwdGlkIjoidXJuOm1hY2U6aW5jb21tb246Y2xlbXNvbi5lZHUhaHR0cHM6Ly9jaWxvZ29uLm9yZy9zaGliYm9sZXRoIVFPTlczeDdvTTB5cStITjkxc0dEcVp3L3BYYz0iLCJhZmZpbGlhdGlvbiI6InN0dWRlbnRAY2xlbXNvbi5lZHU7bWVtYmVyQGNsZW1zb24uZWR1O2VtcGxveWVlQGNsZW1zb24uZWR1IiwibmFtZSI6Ikp1IExpbiIsImFjciI6Imh0dHA6Ly9leGFtcGxlLm9yZy9hYy9jbGFzc2VzL21mYSIsImlzcyI6Imh0dHBzOi8vY2lsb2dvbi5vcmciLCJzdW

In [9]:
status, advertised_topology = orchestrator_proxy.resources(token=fabric_id_token)

print(f"Status: {status}")
print(f"Toplogy: {advertised_topology}")

Status: Status.OK
Toplogy: LBNL: { cpu: 6/6, core: 88/96, ram: 1408/1536G, disk: 108600/109600G, unit: 3/3, }
	Components:
		NVME-P4510:  NVME P4510 { disk: 10000/10000G, unit: 10/10, }
		GPU-RTX6000:  GPU RTX6000 { unit: 2/2, }
		GPU-Tesla T4:  GPU Tesla T4 { unit: 2/4, }
		SharedNIC-ConnectX-6:  SharedNIC ConnectX-6 { unit: 3/3, }
		SmartNIC-ConnectX-6:  SmartNIC ConnectX-6 { unit: 2/2, }
		SmartNIC-ConnectX-5:  SmartNIC ConnectX-5 { unit: 2/2, }
	Site Interfaces:
RENC: { cpu: 6/6, core: 88/96, ram: 1408/1536G, disk: 108600/109600G, unit: 3/3, }
	Components:
		SmartNIC-ConnectX-6:  SmartNIC ConnectX-6 { unit: 2/2, }
		SmartNIC-ConnectX-5:  SmartNIC ConnectX-5 { unit: 0/2, }
		NVME-P4510:  NVME P4510 { disk: 10000/10000G, unit: 10/10, }
		GPU-RTX6000:  GPU RTX6000 { unit: 2/2, }
		GPU-Tesla T4:  GPU Tesla T4 { unit: 4/4, }
		SharedNIC-ConnectX-6:  SharedNIC ConnectX-6 { unit: 3/3, }
	Site Interfaces:
UKY: { cpu: 6/6, core: 92/96, ram: 1472/1536G, disk: 109100/109600G, unit: 3/3, }
	Co

# Create Slice

In [11]:


import fim.user as fu
# Create topology
t = fu.ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='RENC')

# Set capacities
cap = fu.Capacities()
cap.set_fields(core=4, ram=64, disk=500)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# Add PCI devices
n1.add_component(ctype=fu.ComponentType.SmartNIC, model='ConnectX-5', name='nic1')

# Add node
n2 = t.add_node(name='n2', site='UKY')

# Set properties
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# Add PCI devices
n2.add_component(ctype=fu.ComponentType.GPU, model='Tesla T4', name='nic2')

# Add node
n3 = t.add_node(name='n3', site='LBNL')

# Set properties
n3.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# Add PCI devices
n3.add_component(ctype=fu.ComponentType.GPU, model='Tesla T4', name='nic3')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = orchestrator_proxy.create(token=fabric_id_token, slice_name='JupyterSlice1', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

Response Status Status.OK
Reservations created [{
    "graph_node_id": "8664e238-933c-4386-923a-c43ab61ce356",
    "join_state": "None_",
    "lease_end": "2021-04-23 07:00:49",
    "reservation_id": "72594d9a-39e4-4138-a915-41d064a6a0ed",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "922555dc-c65d-4ed5-8b58-8273f934c301"
}, {
    "graph_node_id": "88b5e252-6de7-4888-b305-031cf9e4c130",
    "join_state": "None_",
    "lease_end": "2021-04-23 07:00:49",
    "reservation_id": "91a01f03-41dc-4292-b9cd-5e3e8e0fc296",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "922555dc-c65d-4ed5-8b58-8273f934c301"
}, {
    "graph_node_id": "81454b3f-0618-4922-9764-d731262708e5",
    "join_state": "None_",
    "lease_end": "2021-04-23 07:00:49",
    "reservation_id": "8d295608-3da7-4599-b39f-6fa2ad4c84e9",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "922555dc-c65d-4ed5-8b58-8273f934c301"
}]


In [13]:
slice_id=reservations[0].slice_id
print(slice_id)

922555dc-c65d-4ed5-8b58-8273f934c301


# Renew Slice

In [20]:
Status, renewId = orchestrator_proxy.renew(token=fabric_id_token, slice_id=slice_id, new_lease_end_time="2021-04-22 03:20:00")

In [21]:
status

<Status.OK: 1>

In [22]:
renewId

fabric_cf.orchestrator.swagger_client.rest.ApiException()